## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download("wordnet")
nltk.download('punkt')
nltk.download('stopwords')

## Reading the Dataset

In [2]:
dataset = pd.read_csv('/kaggle/input/twitter-sentiment/Sentiment.csv')

In [3]:
dataset

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [4]:
dataset = dataset[['text','sentiment']]

In [5]:
dataset

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [6]:
dataset = dataset[dataset['sentiment']!= 'Neutral']

In [7]:
dataset

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


## Data Preprocessing

In [8]:
train,test = train_test_split(dataset,test_size=0.1)

In [9]:
train

,text,sentiment
6211,RT @BettyBowers: Trump: A president who talks ...,Negative
7289,RT @tarynoneill: THERE WAS NOT ONE QUESTION ON...,Negative
1671,@GovMikeHuckabee Was fantastic in the debate. ...,Positive
389,"Elections, Democracy, and political debates as...",Negative
13618,RT @RWSurferGirl: I think Cruz and Trump need ...,Positive
...,...,...
750,@YaleClimateComm @JerryBrownGov not a peep fro...,Negative
2920,Megyn Kelly played footies with Debbie Washerw...,Positive
5741,RT @cenkuygur: Love it. Fox News found the one...,Positive
119,RT @palmaceiahome1: Caller on the Rush Limbaug...,Negative


In [10]:
test

,text,sentiment
11570,RT @RWSurferGirl: Why doesn't Chris Wallace as...,Negative
13860,People who say they are #prolife are usually a...,Negative
4146,@riley1999 @megynkelly is a closet democrat; a...,Negative
6978,Gotta give @LindseyGrahamSC props. He saw the ...,Negative
11393,"Dear Donald Trump, sniff a couple times, your ...",Negative
...,...,...
6824,Topics left out of #GOPDebate: 2.) On the 50th...,Negative
4135,RT @JDWinteregg: No GOP POTUS will succeed if ...,Negative
12556,RT @monaeltahawy: Can someone tell me why Ben ...,Negative
71,"RT @HlLLARY: Oh, one more thing. Did anyone he...",Negative


In [11]:
train['text'][9720]

'RT @RWSurferGirl: It is very disappointing that Fox News is not conducting a "Fair &amp; Balanced" Debate. 🇺🇸 #GOPDebate  #GOPDebates'

In [12]:
pattern = "(#\w+)|(RT\s@\w+:)|(http.*)|(@\w+)"

In [13]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [14]:
def Clean_text(data):
    tweets = []
    sentiments = []
    for index,row in data.iterrows():
        sentence = re.sub(pattern,'',row.text)
        words = [e.lower() for e in sentence.split()]
        words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
        words = ' '.join(words)
        tweets.append(words)
        sentiments.append(row.sentiment)
    return tweets,sentiments

In [15]:
train_tweets,train_sentiments = Clean_text(train)

In [16]:
final_data = {'tweets':train_tweets,'sentiments':train_sentiments}

In [17]:
processed_data = pd.DataFrame(final_data)

In [18]:
processed_data

,tweets,sentiments
0,trump: president talk like anonymous internet ...,Negative
1,one question climate change environment.,Negative
2,fantastic debate. white house!,Positive
3,"elections, democracy, political debate enterta...",Negative
4,think cruz trump need band together expose set...,Positive
...,...,...
9651,peep them... take $$ oil &amp; gas! would,Negative
9652,megyn kelly played footies debbie washerwoman ...,Positive
9653,"love it. fox news found one young, black repub...",Positive
9654,"caller rush limbaugh show:""i lost respect fox ...",Negative


In [19]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
processed_data['sentiments'] = labelencoder.fit_transform(processed_data['sentiments'])

In [20]:
processed_data

,tweets,sentiments
0,trump: president talk like anonymous internet ...,0
1,one question climate change environment.,0
2,fantastic debate. white house!,1
3,"elections, democracy, political debate enterta...",0
4,think cruz trump need band together expose set...,1
...,...,...
9651,peep them... take $$ oil &amp; gas! would,0
9652,megyn kelly played footies debbie washerwoman ...,1
9653,"love it. fox news found one young, black repub...",1
9654,"caller rush limbaugh show:""i lost respect fox ...",0


## Converting Words into Vectors

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,3))
cv.fit(processed_data['tweets'])

CountVectorizer(ngram_range=(1, 3))

In [22]:
X_train = cv.transform(processed_data['tweets'])

In [23]:
print(X_train.shape)

(9656, 81872)


In [24]:
! echo 1 > /proc/sys/vm/overcommit_memory

/bin/bash: line 1: /proc/sys/vm/overcommit_memory: Permission denied


In [25]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [26]:
target = processed_data['sentiments'].values

In [27]:
target

array([0, 0, 1, ..., 1, 0, 0])

## Sentiment Analysis (Model Building)

In [28]:
from sklearn.naive_bayes import MultinomialNB 
classifier = MultinomialNB()

In [29]:
classifier.fit(X_train.toarray(),target)

MultinomialNB()

In [30]:
test_tweets,test_sentiments = Clean_text(test)

In [31]:
data_test = {'tweets':test_tweets,'sentiments':test_sentiments}
final_test_data = pd.DataFrame(data_test)

In [32]:
X_test = cv.transform(final_test_data['tweets'])

In [33]:
X_test.shape

(1073, 81872)

In [34]:
y_pred = classifier.predict(X_test.toarray())

In [35]:
final_test_data['sentiments'] = labelencoder.fit_transform(final_test_data['sentiments'])

In [36]:
actual_values = final_test_data['sentiments'].values

In [37]:
actual_values

array([0, 0, 0, ..., 0, 0, 1])

In [38]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,actual_values))

0.8350419384902144


In [39]:
final_test_data

,tweets,sentiments
0,chris wallace ask politicans finance money com...,0
1,people say usually anti-social programs... kno...,0
2,"closet democrat; actually, come closet",0
3,gotta give props. saw ambush calling it.,0
4,"dear donald trump, sniff couple times, head as",0
...,...,...
1068,topic left : 2.) 50th anniversary voting right...,0
1069,gop potus succeed bill reach desk watered-down...,0
1070,someone tell ben carson carrying water people ...,0
1071,"oh, one thing. anyone hear anything affordable...",0


In [40]:
X_test.toarray()[0]

array([0, 0, 0, ..., 0, 0, 0])

In [41]:
classifier.predict(X_test.toarray()[[0]])

array([0])